In [1]:
sc

In [2]:
import sklearn as sk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import zlib
from base64 import b64decode

### parse urls

In [6]:
urls_nums_dict = {}
nums_urls_dict = {}
with open('1_100/urls.txt', 'r') as f:
    for line in f:
        num, url = line.strip().split('\t')
        urls_nums_dict[url] = int(num)
        nums_urls_dict[int(num)] = url
print len(nums_urls_dict)
for num, key in enumerate(nums_urls_dict):
    if num == 0:
        print key, nums_urls_dict[key]
    

5646
0 http://lenta.ru/news/2010/07/09/return/


### create link graph and delete hanging vertex

In [76]:
def parse_doc(line, urls_nums_dict):
    doc_num, doc_base64 = line.strip().split('\t')
    doc_gzip = b64decode(doc_base64)
    doc_html = zlib.decompress(doc_gzip).decode('utf-8')
    soup = BeautifulSoup(doc_html, 'html.parser')
    links = []
    for a in soup.find_all('a', href=True):
        link = a['href']
        #print link
        if link.startswith('/'):
            url = 'http://lenta.ru' + link
            #print url
            if url in urls_nums_dict:
                #url
                #print urls_nums_dict[url]
                if urls_nums_dict[url] not in links: 
                    links.append(urls_nums_dict[url])
    return (int(doc_num), links)

def delHangingVertex(elem, hanging_vertex):
    result = []
    for num_vertex in elem[1]:
        if num_vertex not in hanging_vertex:
            result.append(num_vertex)
    return (elem[0], result)
    
    
input_data = sc.textFile('1_100/docs-000.txt')
urls_nums_dict_var = sc.broadcast(urls_nums_dict)

link_graph = input_data.map(lambda line: parse_doc(line, urls_nums_dict_var.value))
hanging_vertex = link_graph.filter(lambda x: len(x[1]) == 0).keys().collect()

while (len(hanging_vertex) > 0):
    hanging_vertex_var = sc.broadcast(hanging_vertex)
    link_graph = link_graph.filter(lambda x: len(x[1])).map(lambda x: delHangingVertex(x, hanging_vertex_var.value))
    hanging_vertex = link_graph.filter(lambda x: len(x[1]) == 0).keys().collect()

In [ ]:
#link_graph.count()
for item in link_graph.take(100):
    print item

### count pagerank

In [215]:
def flatMapRank(elem):
    result = []
    count_vertex = len(elem[1])
    stream_weight = 1. / count_vertex
    for num_vertex in elem[1]:
        result.append([num_vertex, stream_weight])
    return result

def RankForOtherIter(elem):
    result = []
    count_vertex = len(elem[1][0])
    stream_weight = elem[1][1] * 1. / count_vertex
    for num_vertex in elem[1][0]:
        result.append([num_vertex, stream_weight])
    return result

In [184]:
vertex_count = link_graph.count()
factor = 0.8

In [185]:
print vertex_count, factor

2388 0.8


In [186]:
link_graph = link_graph.cache()
rank = link_graph.flatMap(lambda x: flatMapRank(x)).reduceByKey(lambda a, b: a + b).\
    mapValues(lambda x: (1. - factor) / vertex_count + factor * x).cache()

In [187]:
for i in range(10):
    join_rank_graph = link_graph.join(rank)
    rank = join_rank_graph.flatMap(lambda x: RankForOtherIter(x)).reduceByKey(lambda a, b: a + b).\
        mapValues(lambda x: (1. - factor) / vertex_count + factor * x)

In [188]:
rank = rank.map(lambda a: (a[1], a[0])).sortByKey(ascending=False).map(lambda a: (a[1], a[0])).collect()

In [189]:
for i in range(30):
    print nums_urls_dict[rank[i][0]], rank[i][1]
    

http://lenta.ru/articles/2014/11/03/mertro/ 170.686855938
http://lenta.ru/news/2014/12/23/earnings/ 7.34497813059
http://lenta.ru/news/2014/12/23/hasntdecided/ 5.15434354442
http://lenta.ru/news/2015/01/04/intelwoblogs/ 4.12355139338
http://lenta.ru/news/2014/12/17/shar/ 0.129365856583
http://lenta.ru/conf/zadornov/?qall=yes 0.0862821351386
http://lenta.ru/features/thief/boredomtown/ 0.0862821351386
http://lenta.ru/conf/souzmf/?qall=yes 0.0862821351386
http://lenta.ru/onlines/2014/07/09/netarglive/page4/ 0.0862821351386
http://lenta.ru/conf/etmanov/?qall=yes 0.0862821351386
http://lenta.ru/news/2014/12/18/beltradefail/ 0.0862821351386
http://lenta.ru/news/2014/12/19/arctique/ 0.0862821351386
http://lenta.ru/onlines/2013/11/06/zenpor/page2/ 0.0862821351386
http://lenta.ru/news/2014/12/30/resign/ 0.0858596128221
http://lenta.ru/news/2014/12/25/malaysia/ 0.0188607492904
http://lenta.ru/news/2014/12/30/eyesurgery/ 0.00993847866812
http://lenta.ru/news/2014/12/30/minfin/ 0.00679701051489
ht

## Hits algorithm

### read data

In [245]:
def makeInvGraph(elem):
    result = []
    proccessVertex = elem[0]
    for nowVertex in elem[1]:
        result.append([nowVertex, proccessVertex])
    return result

input_data = sc.textFile('1_100/docs-000.txt')
urls_nums_dict_var = sc.broadcast(urls_nums_dict)

graph = input_data.map(lambda line: parse_doc(line, urls_nums_dict_var.value)).cache()
inv_graph = graph.flatMap(lambda x: makeInvGraph(x)).map(lambda x: (x[0], [x[1]])).\
            reduceByKey(lambda a, b: a + b).cache()

In [246]:
#for item in graph.take(10):
#    print item
for item in inv_graph.take(10):
    print item

(1290, [2983])
(4368, [765])
(798, [2978, 5303])
(1116, [1116])
(2730, [2730])
(306, [306])
(3378, [324])
(5556, [2641, 5556])
(3420, [23, 42, 49, 63, 70, 96, 101, 104, 130, 163, 209, 225, 237, 247, 266, 320, 335, 374, 387, 425, 442, 463, 465, 530, 532, 600, 604, 623, 656, 660, 766, 779, 830, 838, 875, 924, 927, 954, 975, 1013, 1045, 1080, 1161, 1168, 1191, 1265, 1270, 1298, 1314, 1327, 1340, 1350, 1370, 1386, 1439, 1447, 1453, 1476, 1485, 1544, 1555, 1565, 1608, 1673, 1832, 1865, 1889, 1928, 1965, 2019, 2065, 2071, 2080, 2086, 2097, 2098, 2151, 2185, 2188, 2191, 2212, 2229, 2232, 2233, 2238, 2240, 2246, 2256, 2267, 2339, 2341, 2384, 2441, 2465, 2487, 2500, 2523, 2542, 2583, 2595, 2599, 2606, 2615, 2643, 2646, 2654, 2671, 2672, 2723, 2732, 2740, 2742, 2797, 2808, 2819, 2829, 2881, 2887, 2921, 2937, 2966, 2994, 3007, 3073, 3075, 3110, 3136, 3160, 3162, 3296, 3304, 3317, 3340, 3343, 3354, 3367, 3374, 3380, 3384, 3393, 3420, 3551, 3563, 3577, 3589, 3612, 3635, 3652, 3668, 3766, 3779, 3781

### initialize hub and auto for each node

In [247]:
auth_val = graph.map(lambda x: (x[0], 1.)).cache()
hub_val = graph.map(lambda x: (x[0], 1.)).cache()

## cut graph

In [248]:
hanging_vertex = inv_graph.filter(lambda x: len(x[1]) == 0).keys().collect()
while (len(hanging_vertex) > 0):
    hanging_vertex_var = sc.broadcast(hanging_vertex)
    inv_graph = inv_graph.filter(lambda x: len(x[1])).map(lambda x: delHangingVertex(x, hanging_vertex_var.value)).cache()
    hanging_vertex = inv_graph.filter(lambda x: len(x[1]) == 0).keys().collect()

In [249]:
hanging_vertex = graph.filter(lambda x: len(x[1]) == 0).keys().collect()
while (len(hanging_vertex) > 0):
    hanging_vertex_var = sc.broadcast(hanging_vertex)
    graph = graph.filter(lambda x: len(x[1])).map(lambda x: delHangingVertex(x, hanging_vertex_var.value)).cache()
    hanging_vertex = graph.filter(lambda x: len(x[1]) == 0).keys().collect()

### count auto and hub values for each vertex

In [250]:
for i in range(10):
    new_hub_val = inv_graph.join(auth_val).flatMap(lambda x: RankForOtherIter(x)).reduceByKey(lambda a, b:a+b)
    new_auth_val = graph.join(hub_val).flatMap(lambda x: RankForOtherIter(x)).reduceByKey(lambda a, b:a+b)
    #hub_norm_val = np.sqrt(new_hub_val.map(lambda x: x[1]**2).sum())
    #hub_norm_val_sc = sc.broadcast(hub_norm_val)
    #auth_norm_val = np.sqrt(new_auth_val.map(lambda x: x[1] ** 2).sum())
    #auth_norm_val_sc = sc.broadcast(auth_norm_val)
    #new_hub_val = new_hub_val.map(lambda x: (x[0], x[1] / hub_norm_val_sc))
    #new_auth_val = auth_norm_val.map(lambda x: (x[0], x[1] / auth_norm_val_sc))
    auth_val = new_auth_val.cache()
    hub_val = new_hub_val.cache()

### ouput result

In [251]:
auth_val = auth_val.sortBy(lambda x: x[1], ascending=False).collect()
hub_val = hub_val.sortBy(lambda x: x[1], ascending=False).collect()

In [252]:
for i in range(30):
    print nums_urls_dict[auth_val[i][0]], auth_val[i][1]

http://lenta.ru/articles/2014/11/03/mertro/ 47.2377584638
http://lenta.ru/news/2014/12/30/minfin/ 5.19020504027
http://lenta.ru/news/2014/12/31/sportputin/ 4.13819031019
http://lenta.ru/news/2014/12/25/letter1/ 3.94853279504
http://lenta.ru/news/2008/02/22/avalanche/ 3.0
http://lenta.ru/news/2007/05/22/rosenior/ 3.0
http://lenta.ru/news/2014/12/25/malaysia/ 2.7652665992
http://lenta.ru/news/2014/12/16/dmcreissue/ 2.70016718107
http://lenta.ru/news/2014/12/14/lipnizkaya/ 2.0009765625
http://lenta.ru/news/2010/05/11/pass/ 2.0
http://lenta.ru/news/2010/09/16/khryapa/ 2.0
http://lenta.ru/news/2014/10/23/blame/ 2.0
http://lenta.ru/news/2011/04/29/zaitseva1/ 2.0
http://lenta.ru/articles/2010/05/11/resign/ 2.0
http://lenta.ru/news/2014/12/23/earnings/ 1.94084896232
http://lenta.ru/news/2014/12/17/shar/ 1.666015625
http://lenta.ru/photo/2014/08/22/afp/ 1.5
http://lenta.ru/news/2014/12/26/krym/ 1.45474786496
http://lenta.ru/news/2014/12/26/basketball/ 1.42021487241
http://lenta.ru/news/2015/01/

In [253]:
for i in range(30):
    print nums_urls_dict[hub_val[i][0]], hub_val[i][1]

http://lenta.ru/news/2014/12/30/minfin/ 2.55108432161
http://lenta.ru/news/2014/12/31/terrorists_lost/ 2.55069096564
http://lenta.ru/news/2014/12/31/sportputin/ 2.55069096564
http://lenta.ru/news/2014/12/02/coca/ 2.50933214617
http://lenta.ru/2007/05/17 2.33835281932
http://lenta.ru/news/2014/09/12/subway/ 2.00156942005
http://lenta.ru/news/2014/11/18/parking1/ 2.00156942005
http://lenta.ru/iraq/2004/05/26/arab/ 1.99785582938
http://lenta.ru/news/2008/10/21/equador/ 1.99785582938
http://lenta.ru/vybory/2000/03/22/yavlinsky 1.99278565461
http://lenta.ru/news/2005/05/18/amber/ 1.99278565461
http://lenta.ru/news/2006/02/28/george/ 1.99278565461
http://lenta.ru/news/2006/12/19/thai/ 1.99278565461
http://lenta.ru/news/2010/10/22/zoocompens/ 1.99278565461
http://lenta.ru/world/2001/12/28/snow/ 1.99278565461
http://lenta.ru/articles/2008/10/03/gas/ 1.99278565461
http://lenta.ru/news/2011/02/16/superheroes/ 1.99278565461
http://lenta.ru/news/2009/04/21/stop/ 1.99278565461
http://lenta.ru/news/